# Visualize

In [2]:
# big
# scene_1000_2000_1000
# scene_1000_1500_1000

# small 
# scene_1000_2000_150_128_5
# scene_1000_1500_70_128_5


filename = "mexico_1000_20000_1000_128_coverage_33_67.txt"

import pygame
import sys
import numpy as np

from typing import List
import math

import tensorflow as tf
from tensorflow.keras import layers, models, regularizers

resolution = 32
@tf.keras.utils.register_keras_serializable()
class TwoChannelVisibilityCNN(tf.keras.Model):
    def __init__(self, **kwargs):
        super(TwoChannelVisibilityCNN, self).__init__(**kwargs)
        self.conv1 = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu', input_shape=(resolution, resolution, 2))
        self.bn1 = layers.BatchNormalization()
        self.pool1 = layers.MaxPooling2D(pool_size=(2, 2))

        self.conv2 = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')
        self.bn2 = layers.BatchNormalization()
        self.pool2 = layers.MaxPooling2D(pool_size=(2, 2))
        
        self.conv3 = layers.Conv2D(128, kernel_size=3, padding='same', activation='relu')
        self.bn3 = layers.BatchNormalization()
        self.pool3 = layers.MaxPooling2D(pool_size=(2, 2))
        
        
        self.conv4 = layers.Conv2D(256, kernel_size=3, padding='same', activation='relu')
        self.bn4 = layers.BatchNormalization()
        self.pool4 = layers.MaxPooling2D(pool_size=(2, 2))
        
        self.conv5 = layers.Conv2D(256, kernel_size=3, padding='same', activation='relu')
        self.bn5 = layers.BatchNormalization()
        self.pool5 = layers.MaxPooling2D(pool_size=(2, 2))
        
        self.conv6 = layers.Conv2D(128, kernel_size=3, padding='same', activation='relu')
        self.bn6 = layers.BatchNormalization()
        self.pool6 = layers.MaxPooling2D(pool_size=(2, 2))
        

        self.flatten = layers.Flatten()
        
        self.fc2 = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.00075))
        self.fc3 = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.00075))
        
        
        self.dropout = layers.Dropout(0.5)  # Increased dropout to reduce overfitting
        
        self.fc4 = layers.Dense(1, activation='sigmoid')  # Output layer for binary classification



    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.pool2(x)
        
        
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.pool3(x)
        
        
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.pool4(x)
        
        x = self.flatten(x)

        x = self.fc2(x)
        
        x = self.fc3(x)
        return self.fc4(x)

    def get_config(self):
        config = super(TwoChannelVisibilityCNN, self).get_config()
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

def round_down(value: float) -> int:
    # Extract the integer part and the fractional part
    integer_part = int(value)
    fractional_part = value - integer_part

    # Check if the fractional part is less than or equal to 0.5
    if fractional_part <= 0.5:
        return integer_part  # Round down
    else:
        return integer_part + 1  # Round up
    

def bound(value, min_value, max_value):
    
    return max(min(value, max_value), min_value)


class SceneEncoder:
    def __init__(self, minX, maxX, minY, maxY, obstacles):
        self.minX = minX
        self.minY = minY
        self.maxX = maxX
        self.maxY = maxY
        self.obstacles = obstacles

    def getXintersect(self, y, vertices):
        """Get all x-coordinates where the horizontal line at y intersects the polygon defined by vertices."""
        ans = []
        n = len(vertices)
        
        for i in range(n):
            (pointAx,pointAy) = vertices[i]
            (pointBx,pointBy) = vertices[(i + 1) % n]
            
            # Line equation for edge AB: a1*x + b1*y + c1 = 0
            a1 = pointAy - pointBy
            b1 = -(pointAx - pointBx)
            c1 = -pointAx * (pointAy - pointBy) + pointAy * (pointAx - pointBx)
            
            # Line equation for horizontal line: a2*x + b2*y + c2 = 0
            a2, b2, c2 = 0, 1, -y
            
            # Check for parallel lines
            if a1 * b2 - a2 * b1 == 0:
                continue
            
            # Intersection point x
            x = (b1 * c2 - b2 * c1) / (a1 * b2 - a2 * b1)
            
            # Check if x is within the segment bounds
            if (min(pointAx, pointBx) <= x <= max(pointAx, pointBx) and 
                min(pointAy, pointBy) <= y <= max(pointAy, pointBy)):
                ans.append(x)
        
        ans.sort()
        if len(ans)>1:
            if (ans[0]==ans[1]):
                ans = ans[1:]
            if len(ans)>1 and (ans[-1]==ans[-2]):
                ans = ans[:-1]
        return ans

    def encodeScene(self, cellXCount: int, cellYCount: int):
        """Encode the scene into a grid, marking cells with obstacles as True."""
        cellX = (self.maxX - self.minX) / cellXCount
        cellY = (self.maxY - self.minY) / cellYCount
        encodedScene = np.zeros((cellYCount,cellXCount,2))

        for id in self.obstacles:
            # Find min and max y for the obstacle
            minY = min(y for (x,y) in self.obstacles[id])
            maxY = max(y for (x,y) in self.obstacles[id])
            
            # Process each horizontal slice (grid row)
            j = self.minY + round((maxY - self.minY) / cellY) * cellY - cellY / 2.0
            while j >= self.minY + round_down((minY - self.minY) / cellY) * cellY :
                ans = self.getXintersect(j, self.obstacles[id])
                # Process each vertical slice (grid column) within the row
                
                for k in range(0, len(ans)-1, 2):
                    l = self.minX + round_down((ans[k] - self.minX) / cellX) * cellX + cellX / 2.
                    while l <= self.minX + round((ans[k + 1] - self.minX) / cellX) * cellX :
                        x_idx = math.floor((l - self.minX) / cellX)
                        y_idx = math.floor((j - self.minY) / cellY)
                        x_idx, y_idx = bound(x_idx,0,cellXCount-1), bound(y_idx,0,cellYCount-1)
                        encodedScene[y_idx][x_idx][0] = 1
                        l += cellX
                j -= cellY
        
        return encodedScene

def prepare_analysis_dataset(scene,cell):
    scene_dimension = scene[:,:,0].shape
    data = []
    point1 = np.zeros(scene_dimension)
    point1[cell] = 1
    for i in range(scene_dimension[0]):
        for j in range(scene_dimension[1]):
            if (i,j)==cell:
                continue
            point2 = np.copy(point1)
            point2[(i,j)] = 1
            data.append(np.stack([scene[:,:,0], point2], axis=-1))
    
    return np.array(data)


def generate_pygame_data(scene,cell,model):
    prediction = None
    scene_dimension = scene.shape
    index = 0
    updated_scene = scene.copy()
    prediction = (model.predict(prepare_analysis_dataset(scene,cell)) >= 0.5).astype(int)
    for i in range(scene_dimension[0]):
        for j in range(scene_dimension[1]):
            if (i,j)==cell:
                updated_scene[i,j,1] = 2
                continue
            if (prediction[index]==1):
                updated_scene[i,j,1] = 3
            else:
                updated_scene[i,j,1] = 4
            index = index + 1
    return updated_scene

def is_inside(vertex, x_min, x_max, y_min, y_max, edge):
    x, y = vertex
    if edge == 0:  # Left edge: x >= x_min
        return x >= x_min
    elif edge == 1:  # Right edge: x <= x_max
        return x <= x_max
    elif edge == 2:  # Bottom edge: y >= y_min
        return y >= y_min
    elif edge == 3:  # Top edge: y <= y_max
        return y <= y_max
    return False

def intersect(v1, v2, x_min, x_max, y_min, y_max, edge):
    x1, y1 = v1
    x2, y2 = v2
    result = [0, 0]
    if edge == 0:  # Left edge: x = x_min
        slope = (y2 - y1) / (x2 - x1)
        result[0] = x_min
        result[1] = y1 + slope * (x_min - x1)
    elif edge == 1:  # Right edge: x = x_max
        slope = (y2 - y1) / (x2 - x1)
        result[0] = x_max
        result[1] = y1 + slope * (x_max - x1)
    elif edge == 2:  # Bottom edge: y = y_min
        slope = (x2 - x1) / (y2 - y1)
        result[1] = y_min
        result[0] = x1 + slope * (y_min - y1)
    elif edge == 3:  # Top edge: y = y_max
        slope = (x2 - x1) / (y2 - y1)
        result[1] = y_max
        result[0] = x1 + slope * (y_max - y1)
    return tuple(result)

def sutherland_hodgman(poly, x_min, x_max, y_min, y_max):
    input_vertices = poly[:]
    for edge in range(4):  # Iterate through all 4 edges
        output_vertices = []
        for i in range(len(input_vertices)):
            curr = input_vertices[i]
            prev = input_vertices[(i + len(input_vertices) - 1) % len(input_vertices)]

            curr_inside = is_inside(curr, x_min, x_max, y_min, y_max, edge)
            prev_inside = is_inside(prev, x_min, x_max, y_min, y_max, edge)

            if prev_inside and curr_inside:
                output_vertices.append(curr)
            elif prev_inside and not curr_inside:
                output_vertices.append(intersect(prev, curr, x_min, x_max, y_min, y_max, edge))
            elif not prev_inside and curr_inside:
                output_vertices.append(intersect(prev, curr, x_min, x_max, y_min, y_max, edge))
                output_vertices.append(curr)
        input_vertices = output_vertices
    return output_vertices


def read_obstacle_file(filename):
    obstacles = {}
    scenes = []
    with open(filename, 'r') as file:
        for line in file:
            if line.split()[0]=="a":
                type,scene_id,obstacle_id, vertex_id, x, y = line.split()
                scene_id = int(scene_id)
                obstacle_id = int(obstacle_id)
                x, y = float(x), float(y)
                if obstacle_id not in obstacles:
                    obstacles[obstacle_id] = []
                obstacles[obstacle_id].append((x, y))
            elif line.split()[0]=='b':
                type,min_x,max_x,min_y,max_y = line.split()
                min_x, max_x, min_y, max_y = float(min_x), float(max_x), float(min_y), float(max_y)
                # min_x, max_x, min_y, max_y = 0,1,0,1
                scenes.append(SceneEncoder(min_x,max_x,min_y,max_y,obstacles))
                obstacles = {}
    return scenes


def transform_coordinates(vertices, offset, scale, min_x, min_y, screen_height):
    """Transform vertices for Cartesian coordinates (y increases upward)."""
    return [
        (offset[0] + (x - min_x) * scale, 
         screen_height - (offset[1] + (y - min_y) * scale)) 
        for x, y in vertices
    ]

def clamp(value, min_value, max_value):
    """Clamp a value to be within a range."""
    return max(min_value, min(value, max_value))

def get_corner_coordinates(min_x, max_x, min_y, max_y, scale, offset, screen_size):
    # Top-left
    top_left_x = min_x + (0 - offset[0]) / scale
    top_left_y = min_y + (screen_size - offset[1]) / scale
    # Top-right
    top_right_x = min_x + (screen_size - offset[0]) / scale
    top_right_y = min_y + (screen_size - offset[1]) / scale
    # Bottom-left
    bottom_left_x = min_x + (0 - offset[0]) / scale
    bottom_left_y = min_y + (0 - offset[1]) / scale
    # Bottom-right
    bottom_right_x = min_x + (screen_size - offset[0]) / scale
    bottom_right_y = min_y + (0 - offset[1]) / scale

    return [(top_left_x,top_left_y),(top_right_x,top_right_y),(bottom_left_x,bottom_left_y),(bottom_right_x,bottom_right_y)]


def draw_coordinates_on_screen(screen, font, min_x, max_x, min_y, max_y, scale, offset, screen_size):
    """Draw corner coordinates (Cartesian) on the screen."""
       

    [(top_left_x,top_left_y),(top_right_x,top_right_y),(bottom_left_x,bottom_left_y),(bottom_right_x,bottom_right_y)] = get_corner_coordinates(min_x, max_x, min_y, max_y, scale, offset, screen_size)

    # Render coordinates as text
    corners = [
        (f"({top_left_x:.2f}, {top_left_y:.2f})", (10, 10)),  # Top-left
        (f"({top_right_x:.2f}, {top_right_y:.2f})", (screen_size - 180, 10)),  # Top-right
        (f"({bottom_left_x:.2f}, {bottom_left_y:.2f})", (10, screen_size - 30)),  # Bottom-left
        (f"({bottom_right_x:.2f}, {bottom_right_y:.2f})", (screen_size - 180, screen_size - 30)),  # Bottom-right
    ]

    for text, position in corners:
        rendered_text = font.render(text, True, (0, 0, 0))
        screen.blit(rendered_text, position)


def draw_scene_id(screen, font, screen_size, idx):
    rendered_text = font.render(f"{idx}", True, (255, 0, 0))
    screen.blit(rendered_text, (screen_size//2,10))



WHITE = (255, 255, 255)
BLACK = (0,0,0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
GRAY = (200, 200, 200)
background_color = (255, 255, 255)  # White
obstacle_fill_color = (150, 150, 150)  # Gray
obstacle_outline_color = (0, 0, 255)  # Blue

glass_alpha = 20


color_map = {
    0: WHITE,
    1: obstacle_fill_color,
    2: BLUE,
    3: GREEN,
    4: RED
}

def draw_grid(DIS,data,CELL_WIDTH,CELL_HEIGHT,screen_size):
    for row in range(data.shape[0]):
        for col in range(data.shape[1]):
            pygame.draw.rect(DIS, color_map[data[row][col][0]], (col * CELL_WIDTH, screen_size-(row+1) * CELL_HEIGHT, CELL_WIDTH, CELL_HEIGHT))
            # glass_surface = pygame.Surface((CELL_WIDTH, CELL_HEIGHT), pygame.SRCALPHA)
            if (data[row][col][1]>2):
                # glass_surface.fill((*color_map[data[row][col][1]], glass_alpha))
                # DIS.blit(glass_surface, (col * CELL_WIDTH, screen_size- (row+1) * CELL_HEIGHT))
                pygame.draw.rect(DIS, color_map[data[row][col][1]], (col * CELL_WIDTH + (CELL_WIDTH-CELL_WIDTH//4)//2, screen_size- (row+1) * CELL_HEIGHT + (CELL_HEIGHT-CELL_HEIGHT//4)//2, CELL_WIDTH//4, CELL_HEIGHT//4))
            elif (data[row][col][1]==2):
                # glass_surface.fill((*color_map[data[row][col][1]], glass_alpha))
                # DIS.blit(glass_surface, (col * CELL_WIDTH, screen_size- (row+1) * CELL_HEIGHT))
                
                pygame.draw.rect(DIS, color_map[data[row][col][1]], (col * CELL_WIDTH + CELL_WIDTH//4, screen_size- (row+1) * CELL_HEIGHT + CELL_HEIGHT//4, CELL_WIDTH//2, CELL_HEIGHT//2))

def get_cell_under_mouse(mouse_pos,CELL_WIDTH,CELL_HEIGHT,ROWS,COLS):
    x, y = mouse_pos
    col = x // CELL_WIDTH
    row = y // CELL_HEIGHT
    if row < ROWS and col < COLS:
        return (int(row), int(col))
    return

def draw_select(screen,start_pos,end_pos):
    glass_surface = pygame.Surface((abs(start_pos[0]-end_pos[0]), abs(start_pos[1]-end_pos[1])), pygame.SRCALPHA)
    glass_surface.fill((*(100,100,100), 100))
    screen.blit(glass_surface, (min(start_pos[0],end_pos[0]),min(start_pos[1],end_pos[1])))

def main():
    # Initialize pygame
    pygame.init()

    # Screen dimensions (square screen)
    screen_size = 704
    screen = pygame.display.set_mode((screen_size, screen_size))
    pygame.display.set_caption("Obstacle Visualization with Zoom and Pan")

    # Colors
    
    # Load obstacles
    scenes = read_obstacle_file(filename)
    
    idx = 0
    
    # Calculate scene bounds and scale
    min_x, max_x,min_y, max_y = scenes[idx].minX, scenes[idx].maxX, scenes[idx].minY, scenes[idx].maxY
    obstacles = scenes[idx].obstacles
    scene_width = max_x - min_x
    scene_height = max_y - min_y

    scale = screen_size / min(scene_width, scene_height)

    offset = [0, 0]

    # Movement and zoom variables
    zoom_factor = 1.05
    dragging = False
    drag_start = (0, 0)
    move_speed = 5
    moving = {"up": False, "down": False, "left": False, "right": False}
    zooming = {"in": False, "out": False}
    current_scale = scale
    special_mode = False
    right_dragging = False
    start_pos = None
    end_pos = None
    bounding_box = True

    cellCount = [32,64]
    modelFileName = ['../saved_models/corrected_combined_0_50.keras','../saved_models/corrected_v2_0_50_64x64.keras']
    cellCount_idx = 0
    cellXCount = cellYCount = cellCount[cellCount_idx]
    CELL_WIDTH = screen_size/cellXCount
    CELL_HEIGHT = screen_size/cellYCount
    grid = None
    grid_marked = None
    new_obstacles = None
    model = None

    # Main loop
    clock = pygame.time.Clock()
    running = True
    while running:
        mouse_pos = pygame.mouse.get_pos()
        for event in pygame.event.get():
            
            if event.type == pygame.QUIT:
                running = False

            # Key down events
            elif event.type == pygame.KEYDOWN:
                if (event.key == pygame.K_i or event.key == pygame.K_k) and not special_mode:
                    if event.key == pygame.K_i:
                        idx = (idx + 1)%len(scenes)
                    elif event.key == pygame.K_k:
                        idx = (idx - 1)%len(scenes)
                    min_x, max_x, min_y, max_y = scenes[idx].minX, scenes[idx].maxX, scenes[idx].minY, scenes[idx].maxY
                    obstacles = scenes[idx].obstacles
                    scene_width = max_x - min_x
                    scene_height = max_y - min_y

                    scale = screen_size / min(scene_width, scene_height)

                    offset = [0, 0]

                    # Movement and zoom variables
                    zoom_factor = 1.05
                    dragging = False
                    drag_start = (0, 0)
                    move_speed = 5
                    moving = {"up": False, "down": False, "left": False, "right": False}
                    zooming = {"in": False, "out": False}
                    current_scale = scale
                    special_mode = False
                    right_dragging = False
                    start_pos = None
                    end_pos = None
                    bounding_box = True

                if event.key == pygame.K_b and special_mode:  # Right mouse button
                    bounding_box = not bounding_box

                if event.key == pygame.K_e and special_mode:  # Right mouse button
                    cellCount_idx = (cellCount_idx+1)%len(cellCount)
                    cellXCount = cellYCount = cellCount[cellCount_idx]
                    CELL_WIDTH = screen_size/cellXCount
                    CELL_HEIGHT = screen_size/cellYCount
    
                    [(top_left_x,top_left_y),(top_right_x,top_right_y),(bottom_left_x,bottom_left_y),(bottom_right_x,bottom_right_y)] = get_corner_coordinates(min_x, max_x, min_y, max_y, current_scale, offset, screen_size)
                    new_obstacles = {}
                    new_polygon_id = 0
                    for id in obstacles:
                        clipped_vertices =  sutherland_hodgman(obstacles[id], bottom_left_x, bottom_right_x, bottom_left_y, top_left_y)
                        if clipped_vertices:  # Check if the list of vertices is not empty
                            new_obstacles[new_polygon_id] = clipped_vertices
                            new_polygon_id += 1
                    scene = SceneEncoder(bottom_left_x, bottom_right_x, bottom_left_y, top_left_y,new_obstacles)
                    grid = scene.encodeScene(cellXCount,cellYCount)
                    grid_marked = grid.copy()
                    model = tf.keras.models.load_model(modelFileName[cellCount_idx])
                
                if event.key == pygame.K_g and not special_mode:  # Right mouse button
                    start_pos = pygame.mouse.get_pos()
                    end_pos = start_pos
                    right_dragging = True

                if event.key == pygame.K_0:  # Toggle special mode
                    special_mode = not special_mode
                    if special_mode:
                        [(top_left_x,top_left_y),(top_right_x,top_right_y),(bottom_left_x,bottom_left_y),(bottom_right_x,bottom_right_y)] = get_corner_coordinates(min_x, max_x, min_y, max_y, current_scale, offset, screen_size)
                        new_obstacles = {}
                        new_polygon_id = 0
                        for id in obstacles:
                            clipped_vertices =  sutherland_hodgman(obstacles[id], bottom_left_x, bottom_right_x, bottom_left_y, top_left_y)
                            if clipped_vertices:  # Check if the list of vertices is not empty
                                new_obstacles[new_polygon_id] = clipped_vertices
                                new_polygon_id += 1
                        scene = SceneEncoder(bottom_left_x, bottom_right_x, bottom_left_y, top_left_y,new_obstacles)
                        grid = scene.encodeScene(cellXCount,cellYCount)
                        grid_marked = grid.copy()
                        model = tf.keras.models.load_model(modelFileName[cellCount_idx])
                if not special_mode:
                    if event.key == pygame.K_PLUS or event.key == pygame.K_EQUALS:
                        zooming["in"] = True
                    elif event.key == pygame.K_MINUS:
                        zooming["out"] = True
                    elif event.key == pygame.K_UP:
                        moving["up"] = True
                    elif event.key == pygame.K_DOWN:
                        moving["down"] = True
                    elif event.key == pygame.K_LEFT:
                        moving["left"] = True
                    elif event.key == pygame.K_RIGHT:
                        moving["right"] = True

            # Key up events
            elif event.type == pygame.KEYUP:
                if not special_mode:
                    if event.key == pygame.K_PLUS or event.key == pygame.K_EQUALS:
                        zooming["in"] = False
                    elif event.key == pygame.K_MINUS:
                        zooming["out"] = False
                    elif event.key == pygame.K_UP:
                        moving["up"] = False
                    elif event.key == pygame.K_DOWN:
                        moving["down"] = False
                    elif event.key == pygame.K_LEFT:
                        moving["left"] = False
                    elif event.key == pygame.K_RIGHT:
                        moving["right"] = False
                    elif event.key == pygame.K_g:
                        print("h")
                        end_pos = pygame.mouse.get_pos()
                        right_dragging = False
                        x,y = min(end_pos[0],start_pos[0]), max(end_pos[1],start_pos[1])
                        max_moved = min(abs(end_pos[0] - start_pos[0]), abs(end_pos[1] - start_pos[1]))
                        if (max_moved!=0):
                            print("up")
                            [(top_left_x,top_left_y),(top_right_x,top_right_y),(bottom_left_x,bottom_left_y),(bottom_right_x,bottom_right_y)] = get_corner_coordinates(min_x, max_x, min_y, max_y, current_scale, offset, screen_size)
                            current_scale = screen_size/((max_moved/screen_size)*(top_right_x-top_left_x))
                            x_actual, y_actual = (x/screen_size)*(top_right_x-top_left_x) + top_left_x , ((screen_size-y)/screen_size)*(top_left_y-bottom_left_y)+bottom_left_y
                            offset_x = -(x_actual - min_x) * current_scale 
                            offset_y = 1 - (y_actual - min_y) * current_scale
                            # current_scale = new_scale
                            offset[0] = offset_x
                            offset[1] = offset_y

            # Mouse drag
            elif event.type == pygame.MOUSEBUTTONDOWN and not special_mode:
                if event.button == 1:  # Left mouse button
                    dragging = True
                    drag_start = event.pos
                
            elif event.type == pygame.MOUSEBUTTONDOWN and special_mode:
                if event.button == 1:  # Left mouse button
                    mouse_pos = pygame.mouse.get_pos()
                    clicked_cell = get_cell_under_mouse(mouse_pos,CELL_WIDTH,CELL_HEIGHT,cellYCount,cellXCount)
                    clicked_cell = (cellYCount-1-clicked_cell[0],clicked_cell[1])
                    if clicked_cell:
                        grid_marked = generate_pygame_data(grid,clicked_cell,model)
            elif event.type == pygame.MOUSEBUTTONUP and not special_mode:
                if event.button == 1:  # Left mouse button
                    dragging = False
                
            elif event.type == pygame.MOUSEMOTION and not special_mode:
                if dragging:
                    dx, dy = event.rel
                    offset[0] += dx
                    offset[1] -= dy
                elif right_dragging:
                    end_pos = pygame.mouse.get_pos()

            # Mouse wheel
            elif event.type == pygame.MOUSEWHEEL and not special_mode:
                if event.y > 0:
                    zooming["in"] = True
                elif event.y < 0:
                    zooming["out"] = True

        # Handle continuous zoom
        if zooming["in"]:
            mouse_world_x = (mouse_pos[0] - offset[0]) / current_scale
            mouse_world_y = ((screen_size-mouse_pos[1]) - offset[1]) / current_scale
            current_scale = current_scale * zoom_factor
            offset[0] -= mouse_world_x * (zoom_factor - 1) * current_scale
            offset[1] -= mouse_world_y * (zoom_factor - 1) * current_scale

        if zooming["out"]:
            mouse_world_x = (mouse_pos[0] - offset[0]) / current_scale
            mouse_world_y = ((screen_size-mouse_pos[1]) - offset[1]) / current_scale
            current_scale = max(current_scale / zoom_factor , scale)
            # offset[0] += mouse_world_x * (1 - 1 / zoom_factor) * current_scale
            # offset[1] += mouse_world_y * (1 - 1 / zoom_factor) * current_scale
            offset[0] += mouse_world_x * (zoom_factor - 1) * current_scale
            offset[1] += mouse_world_y * (zoom_factor - 1) * current_scale

        # Clamp offset to stay within the initial scene
        offset[0] = clamp(offset[0], screen_size - scene_width * current_scale, 0)
        offset[1] = clamp(offset[1], screen_size - scene_height * current_scale, 0)

        # Handle continuous movement
        if moving["up"]:
            offset[1] = clamp(offset[1] - move_speed, screen_size - scene_height * current_scale, 0)
        if moving["down"]:
            offset[1] = clamp(offset[1] + move_speed, screen_size - scene_height * current_scale, 0)
        if moving["left"]:
            offset[0] = clamp(offset[0] + move_speed, screen_size - scene_width * current_scale, 0)
        if moving["right"]:
            offset[0] = clamp(offset[0] - move_speed, screen_size - scene_width * current_scale, 0)

        # Clear screen
        screen.fill(background_color)
        # Draw obstacles
        if special_mode:
            screen.fill(WHITE)

            # Draw the grid with the initial colors
            draw_grid(screen,grid_marked,CELL_WIDTH,CELL_HEIGHT,screen_size)

            # Get the mouse position and check which cell is hovered
            mouse_pos = pygame.mouse.get_pos()
            hovered_cell = get_cell_under_mouse(mouse_pos,CELL_WIDTH,CELL_HEIGHT,cellYCount,cellXCount)

            if hovered_cell:
                row, col = hovered_cell
                pygame.draw.rect(screen, GRAY, (col * CELL_WIDTH, row * CELL_HEIGHT, CELL_WIDTH, CELL_HEIGHT))

            if bounding_box:
                for obstacle_id, vertices in new_obstacles.items():
                    transformed_vertices = transform_coordinates(vertices, offset, current_scale, min_x, min_y,screen_size)
                    pygame.draw.polygon(screen, obstacle_outline_color, transformed_vertices, width=2)

        else:
            for obstacle_id, vertices in obstacles.items():
                transformed_vertices = transform_coordinates(vertices, offset, current_scale, min_x, min_y,screen_size)
                pygame.draw.polygon(screen, obstacle_fill_color, transformed_vertices)
                pygame.draw.polygon(screen, obstacle_outline_color, transformed_vertices, width=2)
                # Calculate and display dynamic corner coordinates
            draw_coordinates_on_screen(screen, pygame.font.SysFont("Arial", 18), min_x, max_x, min_y, max_y, current_scale, offset, screen_size)
            draw_scene_id(screen, pygame.font.SysFont("Arial", 20), screen_size, idx)
            if right_dragging:
                draw_select(screen,start_pos,end_pos)

        
        # Update display
        pygame.display.flip()
        clock.tick(60)

    pygame.quit()
    sys.exit()

if __name__ == "__main__":
    main() 


h
up


SystemExit: 

In [ ]:

# intersect 3 on 